<a href="https://www.kaggle.com/code/trakiet/duachuot?scriptVersionId=274445868" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/user-course-final/uc_3m4_enroll_start_end_remain.csv
/kaggle/input/mooccubex-dataset/course-teacher.txt
/kaggle/input/mooccubex-dataset/course.json
/kaggle/input/mooccubex-dataset/paper.json
/kaggle/input/mooccubex-dataset/reply.json
/kaggle/input/mooccubex-dataset/concept-comment.txt
/kaggle/input/mooccubex-dataset/user-video.json
/kaggle/input/mooccubex-dataset/course-comment.txt
/kaggle/input/mooccubex-dataset/concept-problem.txt
/kaggle/input/mooccubex-dataset/user-reply.txt
/kaggle/input/mooccubex-dataset/school.json
/kaggle/input/mooccubex-dataset/translated_reply.json
/kaggle/input/mooccubex-dataset/video.json
/kaggle/input/mooccubex-dataset/concept-video.txt
/kaggle/input/mooccubex-dataset/other.json
/kaggle/input/mooccubex-dataset/course-school.txt
/kaggle/input/mooccubex-dataset/problem.json
/kaggle/input/mooccubex-dataset/user-comment.txt
/kaggle/input/mooccubex-dataset/user.json
/kaggle/input/mooccubex-dataset/exercise-problem.txt
/kaggle/input/mooccubex-datas

In [2]:
BASE_DIR = '/kaggle/input/mooccubex-dataset/'
df_user_course_path = '/kaggle/input/user-course-final/uc_3m4_enroll_start_end_remain.csv'

In [3]:
import pyspark
from pyspark.sql import SparkSession
import warnings
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
import seaborn as sns
import matplotlib.pyplot as plt
from os import truncate
from pyspark.sql.functions import explode, col, sum, from_unixtime, hour, dayofweek, count, round, when, isnull, split, avg, regexp_replace
warnings.filterwarnings("ignore")

spark = SparkSession.builder \
  .appName("reply_user_video_analysis") \
  .master("local[*]") \
  .config("spark.some.config.option", "some-value") \
  .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/08 07:26:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!ls {BASE_DIR}

comment-reply.txt    course-school.txt	      translated_reply.json
concept-comment.txt  course-teacher.txt       user-comment.txt
concept.json	     exercise-problem.txt     user.json
concept-other.txt    other.json		      user-problem.json
concept-paper.txt    paper.json		      user-reply.txt
concept-problem.txt  problem.json	      user-video.json
concept-reply.json   reply.json		      user-xiaomu.json
concept-video.txt    reply-reply.txt	      video_id-ccid.txt
course-comment.txt   school.json	      video.json
course-field.json    teacher.json
course.json	     translated_comment.json


In [5]:
df_user_video=spark.read.json(BASE_DIR + "/user-video.json")
# df_user_video.show()

In [6]:
df_user_video.printSchema()

root
 |-- seq: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- segment: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- end_point: double (nullable = true)
 |    |    |    |    |-- local_start_time: long (nullable = true)
 |    |    |    |    |-- speed: double (nullable = true)
 |    |    |    |    |-- start_point: double (nullable = true)
 |    |    |-- video_id: string (nullable = true)
 |-- user_id: string (nullable = true)



In [7]:
from pyspark.sql.functions import col, explode, from_json, expr
from pyspark.sql.types import StructType, StructField, StringType, ArrayType

# Giải nén `seq` để mỗi dòng có một `video_id`
df_flatten = df_user_video.withColumn("seq", explode(col("seq")))

# Giải nén tiếp `segment` để mỗi dòng là một đoạn xem video
df_flatten = df_flatten.withColumn("segment", explode(col("seq.segment")))

# Chọn các cột cần thiết
df_flatten = df_flatten.select(
    col("user_id"),
    col("seq.video_id").alias("video_id"),
    col("segment.start_point"),
    col("segment.end_point"),
    col("segment.speed"),
    col("segment.local_start_time")
)
# df_flatten.show(truncate=True)

In [8]:
from pyspark.sql.functions import from_unixtime

df_flatten = df_flatten.withColumn("local_start_time", from_unixtime("local_start_time", "yyyy-MM-dd HH:mm:ss"))
# df_flatten.show(truncate=False)

In [9]:
df_flatten = df_flatten.filter(col("start_point") <= col("end_point"))


In [10]:
# df_flatten.count()

**video_duration:** Tổng thời gian dài nhất mà video đã được xem (last_end_time). Đây là thời gian tối đa của video mà người dùng đã từng xem.<br>
**rewind_time:**	Tổng thời gian người dùng tua lại video (khi start_point của đoạn hiện tại nhỏ hơn end_point của đoạn trước đó).<br>
**fast_forward_time:**	Tổng thời gian người dùng tua tới trong video (khi start_point của đoạn hiện tại lớn hơn end_point của đoạn trước đó).<br>
**first_local_start_time:**	Thời điểm đầu tiên mà người dùng bắt đầu xem video (local_start_time của lần xem đầu tiên).<br>
**actual_watch_time:**	Tổng thời gian thực tế mà người dùng đã xem video (tổng các đoạn end_point - start_point). Không tính việc video bị tua nhanh hay chậm.<br>
**weighted_watch_time:**	Tổng thời gian xem có tính tốc độ (lấy mỗi đoạn thời gian thực tế chia cho tốc độ tương ứng, rồi cộng lại). <br>Công thức: \
weighted_watch_time = **sum(**(end_point - start_point) / speed**)**  . 	<br>
Giá trị này phản ánh thời gian video đã chạy nếu xem với tốc độ gốc (1.0x).
**average_speed:**	Tốc độ xem trung bình của người dùng trên video. <br>
Công thức: *actual_watch_time / weighted_watch_time.* <br>
Nếu bằng 1.0, người dùng xem với tốc độ gốc. Nếu lớn hơn 1.0, người dùng xem nhanh hơn. Nếu nhỏ hơn 1.0, người dùng xem chậm lại.

In [11]:
from pyspark.sql.functions import col, min, max, count, sum, lag, first, when, countDistinct

from pyspark.sql.window import Window

# Tạo cửa sổ để so sánh với điểm trước đó của user trên cùng video
window_spec = Window.partitionBy("user_id", "video_id").orderBy("local_start_time")

# Tạo cột prev_end_point để so sánh với điểm trước đó của video
df_lagged = df_flatten.withColumn("prev_end_point", lag("end_point").over(window_spec)) \
                      .withColumn("prev_speed", lag("speed").over(window_spec)) \
                      .withColumn("prev_start_time", lag("local_start_time").over(window_spec))
# Tính toán các giá trị mong muốn
df_user_video_final = df_lagged.groupBy("user_id", "video_id").agg(
    round(max("end_point"), 2).alias("max_watch_point"),      # Tổng thời gian của video (giá trị lớn nhất của end_point)
    sum(
        when(col("prev_end_point") > col("start_point"),
             round(col("prev_end_point") - col("start_point"), 2))  # Nếu prev_end > start -> tua lại
        .otherwise(0)
    ).alias("rewind_time"),
    sum(
        when(col("start_point") > col("prev_end_point"),
             round(col("start_point") - col("prev_end_point"), 2))  # Nếu start > prev_end -> tua tới
        .otherwise(0)
    ).alias("fast_forward_time"),
    first("local_start_time").alias("first_local_start_time"),  # Lần đầu tiên xem video
    round(sum(col("end_point") - col("start_point")), 2).alias("actual_watch_time"),  # Tổng thời gian thực tế đã xem
    round(sum((col("end_point") - col("start_point")) / col("speed")), 2).alias("weighted_watch_time"),  # Tổng thời gian thực / speed

    # --- CÁC ĐẶC TRƯNG MỚI ĐỀ XUẤT ---
    
    # 1. Tần suất và Phiên xem: Ý nghĩa: Đếm tổng số "đoạn" log (rows) cho mỗi (user, video). Một con số cao có thể cho thấy người dùng xem video nhiều lần, hoặc tạm dừng/tua rất nhiều.
    count("*").alias("interaction_count"),
    (sum(when(col("local_start_time").cast("long") - col("prev_start_time").cast("long") > 1800, 1).otherwise(0)) + 1).alias("session_count"),
    # 2. Hành vi Tốc độ: Đếm số phiên xem riêng biệt. Ta có thể định nghĩa một "phiên mới" là khi thời gian giữa hai lần tương tác liên tiếp vượt quá một ngưỡng (ví dụ: 30 phút).
    sum(when(col("speed") != col("prev_speed"), 1).otherwise(0)).alias("speed_change_count"),
    countDistinct("speed").alias("distinct_speeds_count"),

    # 3. Yếu tố Thời gian: Khoảng thời gian từ lần tương tác đầu tiên đến lần tương tác cuối cùng của video đó. Nó cho biết video này đã được xem trong một khoảng thời gian dài (ví dụ: vài ngày) hay xem liền một mạch.
    hour(first("local_start_time")).alias("start_hour"),
    dayofweek(first("local_start_time")).alias("start_dayofweek")
)

# Tính tốc độ xem trung bình
# 3. Tính các đặc trưng phái sinh (Post-Aggregation)
df_user_video_final = df_user_video_final.withColumn(
    "average_speed", 
    round(col("actual_watch_time") / col("weighted_watch_time"), 2)
)
df_user_video_final.show(truncate=True)


+----------+---------+---------------+------------------+------------------+----------------------+-----------------+-------------------+-----------------+-------------+------------------+---------------------+----------+---------------+-------------+
|   user_id| video_id|max_watch_point|       rewind_time| fast_forward_time|first_local_start_time|actual_watch_time|weighted_watch_time|interaction_count|session_count|speed_change_count|distinct_speeds_count|start_hour|start_dayofweek|average_speed|
+----------+---------+---------------+------------------+------------------+----------------------+-----------------+-------------------+-----------------+-------------+------------------+---------------------+----------+---------------+-------------+
|U_10001587|V_6043067|         6123.8|               0.0|               0.0|   2020-08-18 09:33:38|            613.8|              306.9|                1|            1|                 0|                    1|         9|              3|       

In [12]:
df_user_video_final.count()

1977141

In [13]:
uv_null_ratio = df_user_video_final.select([
    (F.count(F.when(F.col(c).isNull(), 1))).alias(c) for c in df_user_video_final.columns
])
# uv_null_ratio.show()

In [14]:
df_course = spark.read.json(BASE_DIR + '/course.json')
df_course.show(truncate=True)

+-------------------------------------+------------------------------+--------+--------------------------------+-------------+--------------------+
|                                about|                         field|      id|                            name|prerequisites|            resource|
+-------------------------------------+------------------------------+--------+--------------------------------+-------------+--------------------+
|通过老师导读，同学们可深入这一经典...|        [历史学, 中国语言文学]|C_584313|                《资治通鉴》导读|             |[{1.1.1, V_849, [...|
|本课程是理工科的一门数学基础课，系...|[应用经济学, 数学, 物理学, ...|C_584329|      微积分——极限理论与一元函数|             |[{1.1.1, V_1350, ...|
|掌握基本的摄影技能，了解图片新闻的...|          [艺术学, 新闻传播学]|C_584381|                        新闻摄影|             |[{1.1.1, V_1800, ...|
|  最有趣的理论+最有用的算法=不得不...|            [计算机科学与技术]|C_597208|            数据挖掘：理论与算法|             |[{1.1.1, V_2961, ...|
|大学计算机课程将以计算思维为导向，...|                            []|C_597225|                      大学计算机|         

In [15]:
df_video_course_map = (
    df_course
    .withColumn(
        "video_resource",
        F.filter(F.col("resource"), lambda x: x["resource_id"].startswith("V_"))
    )
    .withColumn("video_struct", F.explode_outer(F.col("video_resource")))
    .select(
        F.col("id").alias("course_id"),
        F.col("video_struct.resource_id").alias("video_id")
    )
)

# df_video_course_map.show()

In [16]:
df_video = spark.read.json(BASE_DIR + "/video.json")
# df_video.show()

In [17]:
from pyspark.sql.functions import array_max
df_video = (
    df_video
    .select(
        "ccid", 
        array_max("end").alias("video_duration")
    )
)


In [18]:
# df_video.show()

In [19]:
df_video_id_ccid = spark.read.text(BASE_DIR + '/video_id-ccid.txt')
# Parse mapping file
df_video_id_ccid_v2 = df_video_id_ccid.select(
    split(col("value"), "\t")[0].alias("video_id"),
    split(col("value"), "\t")[1].alias("ccid")
)
# df_video_id_ccid_v2.show(5, truncate=False)

In [20]:
video_info = df_video_id_ccid_v2.join(df_video, on=['ccid'])
# video_info.show(truncate=False)


In [21]:
course_video_info = df_video_course_map.join(video_info, 'video_id', 'left')

In [22]:
course_video_info = course_video_info.drop('ccid')
# course_video_info.show()


In [23]:
from pyspark.sql import functions as F

course_agg = (
    course_video_info
    .filter(F.col("video_id").isNotNull())  # bỏ video_id NULL
    .groupBy("course_id")
    .agg(
        F.collect_list("video_id").alias("video_list"),
        F.coalesce(F.sum("video_duration"), F.lit(0.0)).alias("total_video_duration"),
        F.count("video_id").alias("num_videos")  # chỉ đếm video hợp lệ
    )
)

# course_agg.filter(F.col("num_videos") == 1).show(truncate=False)


In [24]:
df_user_course = spark.read.csv(df_user_course_path, header=True, inferSchema=True)
df_user_course.show()

+-------+---------+-----------+----------+----------+----------+
|user_id|course_id|enroll_date|start_date|  end_date|remain_day|
+-------+---------+-----------+----------+----------+----------+
|   U_24| C_707038| 2019-10-13|2019-09-09|2020-01-12|        91|
|   U_24| C_696994| 2019-10-24|2019-09-09|2020-01-12|        80|
|   U_24| C_879040| 2019-12-25|2019-10-09|2020-02-29|        66|
|   U_24| C_681655| 2020-01-31|2020-01-27|2020-07-26|       177|
|   U_24| C_697288| 2020-02-16|2020-01-27|2020-07-26|       161|
|   U_24| C_707456| 2020-02-19|2020-02-17|2020-07-19|       151|
|   U_24| C_696946| 2020-04-11|2020-04-01|2020-07-26|       106|
|   U_24|C_1903985| 2020-08-21|2020-08-01|2020-11-30|       101|
|   U_25|C_1903985| 2020-08-07|2020-08-01|2020-11-30|       115|
|   U_53| C_696679| 2020-03-01|2020-01-19|2020-07-31|       152|
|   U_69| C_679390| 2019-09-23|2019-09-09|2020-01-12|       111|
|   U_69| C_696994| 2019-10-12|2019-09-09|2020-01-12|        92|
|   U_90| C_676664| 2020-

In [25]:
df_user_course.count()

3405404

In [26]:
# count = df_user_course_clean.select([
#     (F.count(F.when(F.col(c).isNull(), 1))).alias(c) for c in df_user_course_clean.columns
# ])
# count.show(truncate=False)

In [27]:
df_user_course_group = df_user_course.join(course_agg, 'course_id', 'left')

In [28]:
df_user_course_group.show()

+---------+-------+-----------+----------+----------+----------+--------------------+--------------------+----------+
|course_id|user_id|enroll_date|start_date|  end_date|remain_day|          video_list|total_video_duration|num_videos|
+---------+-------+-----------+----------+----------+----------+--------------------+--------------------+----------+
| C_883345|  U_105| 2020-04-26|2020-01-20|2020-07-31|        96|[V_1007135, V_100...|  254558.64200000005|      1006|
| C_879040|   U_24| 2019-12-25|2019-10-09|2020-02-29|        66|[V_1125916, V_112...|           51429.606|        72|
| C_696994|   U_24| 2019-10-24|2019-09-09|2020-01-12|        80|[V_492398, V_4924...|           29141.432|        80|
| C_696994|   U_69| 2019-10-12|2019-09-09|2020-01-12|        92|[V_492398, V_4924...|           29141.432|        80|
| C_696994|  U_105| 2020-02-25|2020-01-17|2020-07-31|       157|[V_492398, V_4924...|           29141.432|        80|
| C_696994|  U_112| 2019-09-20|2019-09-09|2020-01-12|   

In [29]:
df_user_course_group.count()


3405404

In [31]:
from pyspark.sql import functions as F


In [32]:
df_user_course_group_output = 'df_user_course_group.parquet'
df_user_course_group.coalesce(10).write.mode("overwrite").option("header", "true").parquet(df_user_course_group_output)
print("Saved Successfully")

Saved Successfully


In [33]:
import shutil

input_dir = "/kaggle/working/df_user_course_group.parquet"
output_zip = "/kaggle/working/df_user_course_group.parquet"

shutil.make_archive(output_zip, 'zip', input_dir)
print("✅ Đã nén file thành df_user_course_group.parquet.zip")


✅ Đã nén file thành df_user_course_group.parquet.zip


In [ ]:
# df_user_video_final_output = 'df_user_video_final.parquet'
# df_user_video_final.coalesce(1).write.mode("overwrite").option("header", "true").parquet(df_user_video_final_output)
# print("Saved Successfully")

In [ ]:
# import shutil

# input_dir = "/kaggle/working/df_user_video_final.parquet"
# output_zip = "/kaggle/working/df_user_video_final.parquet"

# shutil.make_archive(output_zip, 'zip', input_dir)
# print("✅ Đã nén file thành df_user_video_final.parquet.zip")


In [ ]:
# rm -rf /kaggle/working/*


In [ ]:
# B1. Explode video_list để mỗi video_id trong course thành 1 dòng
df_course_exploded = (
    df_user_course_group
    .withColumn("video_id", F.explode("video_list"))
    .select(
        "course_id",
        "user_id",
        "video_id",
        "certificate",
        "start_date",
        "end_date",
        "enroll_time",
        "duration_days",
        "remain_day",
        "total_video_duration",
        "num_videos"
    )
)

In [ ]:
df_joined_full = (
    df_course_exploded
    .join(
        df_user_video_final,
        on=["user_id", "video_id"],
        how="left"      # quan trọng!
    )
)

In [ ]:
df_joined_full.count()

In [ ]:
df_joined.count() 


In [ ]:
# --- 2. Thực hiện phép JOIN cuối cùng của bạn (nguyên nhân gây lỗi) ---
# Bây giờ phép join này sẽ chạy trên df_with_stats_fixed đã được tối ưu
df_final = df_with_stats_fixed.join(
    df_user_video_final, 
    ['user_id', 'video_id'], 
    'left'
)

# --- 3. (Khuyến nghị) Xử lý các giá trị NULL như ở phần trước ---
# Các video được đăng ký nhưng "chưa xem" sẽ có giá trị NULL
# Ta điền 0 vào các cột hành vi
behavior_cols = [
    'max_watch_point', 'rewind_time', 'fast_forward_time', 'actual_watch_time',
    'weighted_watch_time', 'interaction_count', 'session_count', 'speed_change_count',
    'distinct_speeds_count', 'average_speed'
]

df_final = df_final.fillna(0, subset=behavior_cols)

# Gán tốc độ 1.0 cho các video chưa xem
df_final = df_final.withColumn(
    "average_speed",
    F.when(F.col("actual_watch_time") == 0, 1.0) 
    .otherwise(F.col("average_speed"))
)


# --- 4. Hiển thị kết quả (Bây giờ sẽ chạy được) ---
print("Đã join thành công, đang hiển thị kết quả:")
# df_final.show()

In [ ]:
# check = df_final.filter(F.col('video_duration') > 0)
# check.show(200, truncate=False)

In [ ]:
# from pyspark.sql import functions as F
# from pyspark.sql.functions import col, datediff, to_date
# df_with_day = df_final.withColumn(
#     "day_from_enroll", 
#     datediff(to_date(col('first_local_start_time')), col('enroll_time'))
# )

# df_with_abs = df_with_day.withColumn(
#     "day_from_enroll",
#     F.abs(F.col('day_from_enroll'))
# )

# df_final = df_with_abs.drop("first_local_start_time", "enroll_time")

In [ ]:
# # Sửa lỗi chính tả ở đây: "vides_watched_in_course" -> "videos_watched_in_course"
# df_grouped = df_final.groupBy("user_id", "course_id").agg(
#     # Đếm số lượng video mà người học xem được
#     F.countDistinct("video_id").alias("videos_watched_in_course"), # <--- ĐÃ SỬA
#     # Tính tổng thời lượng thực tế của người dùng trong khóa học
#     F.sum('actual_watch_time').alias("total_watch_time_in_course"),
#     # Lấy các giá trị tổng của khóa học 
#     F.first('course_total_videos').alias('course_total_videos'),
#     F.first('course_total_duration_secs').alias('course_total_duration_secs')
# )

# df_grouped.show()

In [ ]:
# # Đoạn code này bây giờ sẽ chạy đúng vì tên cột đã khớp
# df_user_completion = df_grouped.withColumn(
#     # Tỷ lệ hoàn thành dựa trên số lượng video
#     "video_completion_rate",
#     F.when(
#         F.col("course_total_videos") > 0,
#         F.col("videos_watched_in_course") / F.col("course_total_videos") # <--- Giờ đã khớp
#     ).otherwise(0) # Tránh lỗi chia cho 0
# ).withColumn(
#     # Tỷ lệ hoàn thành dựa trên thời lượng xem
#     "duration_completion_rate",
#     F.when(
#         F.col("course_total_duration_secs") > 0,
#         F.col("total_watch_time_in_course") / F.col("course_total_duration_secs")
#     ).otherwise(0) # Tránh lỗi chia cho 0
# )

# # (Tùy chọn) Drop các cột trung gian
# df_user_completion = df_user_completion.drop(
#     'videos_watched_in_course',
#     'course_total_videos',
#     'total_watch_time_in_course',
#     'course_total_duration_secs',
# )

In [ ]:
# df_user_completion.show()

# Thử Nghiệm 

In [ ]:
# from pyspark.sql import functions as F
# from pyspark.sql.types import ArrayType, StringType, TimestampType

# # 1. Chuẩn bị Dữ liệu Video (Không đổi)
# df_video_durations = video_info_cleaned.select(
#     F.col("video_id"),
#     F.col("video_duration")
# ).distinct()

# # 2. Chuẩn bị Dữ liệu Khóa học (Không đổi)
# df_course_video_details = df_video_course_map_clean.join(
#     df_video_durations,
#     "video_id",
#     "left"
# )
# df_course_stats = df_course_video_details.groupBy("course_id").agg(
#     F.collect_list("video_id").alias("course_video_list"),
#     F.sum("video_duration").alias("total_video_duration"),
#     F.count("video_id").alias("num_videos_in_course")
# )

# # 3. Chuẩn bị Dữ liệu Xem của Người dùng-Khóa học
# df_user_watch_by_course = df_user_video_final.join(
#     df_video_course_map_clean,
#     "video_id",
#     "inner"
# )

# # --- BẮT ĐẦU TINH CHỈNH 1 ---
# # Đổi tên 'video_duration' (từ log) để tránh xung đột
# # Cột này (từ user-video.json) đáng tin cậy hơn để tính tỷ lệ hoàn thành sự kiện
# df_user_watch_events = df_user_watch_by_course.withColumnRenamed("video_duration", "log_video_duration")

# # Join với duration "chính thức" (từ video.json)
# df_user_watch_events = df_user_watch_events.join(
#     df_video_durations,
#     "video_id",
#     "left"
# )

# # Tính 'video_completion_rate_event' DÙNG 'log_video_duration' (đáng tin cậy hơn)
# df_user_watch_events = df_user_watch_events.withColumn(
#     "video_completion_rate_event",
#     F.when(F.col("log_video_duration") > 0, F.col("actual_watch_time") / F.col("log_video_duration"))
#      .otherwise(0)
# ).withColumn(
#     "is_rewatch",
#     F.when(F.col("rewind_time") > 0, 1).otherwise(0)
# )
# # --- KẾT THÚC TINH CHỈNH 1 ---


# # Tổng hợp hành vi của người dùng (Không đổi)
# df_user_course_watch_stats = df_user_watch_events.groupBy("user_id", "course_id").agg(
#     F.collect_list("video_id").alias("user_video_list_with_duplicates"),
#     F.sum("actual_watch_time").alias("total_watched_duration"),
#     F.avg("actual_watch_time").alias("avg_watched_duration"),
#     F.avg("average_speed").alias("avg_speed"),
#     F.count("video_id").alias("num_sessions"),
#     F.countDistinct("video_id").alias("num_videos_watched"),
#     F.sum("is_rewatch").alias("num_rewatches"),
#     F.avg("video_completion_rate_event").alias("avg_completion_rate_per_video"), # (Giờ sẽ tính đúng)
#     F.min("first_local_start_time").alias("first_local_start_time")
# ).withColumn(
#     "user_video_list", F.array_distinct("user_video_list_with_duplicates")
# ).drop("user_video_list_with_duplicates")


# # 4. Gộp và Tính toán (Bước cuối cùng)
# df_final_agg = df_user_course_clean.join(
#     df_course_stats,
#     "course_id",
#     "left"
# ).join(
#     df_user_course_watch_stats,
#     ["user_id", "course_id"],
#     "left"
# )

# # Xử lý giá trị NULL (Không đổi)
# numeric_fill_cols = {
#     "total_video_duration": 0.0,
#     "num_videos_in_course": 0,
#     "total_watched_duration": 0.0,
#     "avg_watched_duration": 0.0,
#     "avg_speed": 0.0,
#     "num_sessions": 0,
#     "num_videos_watched": 0,
#     "num_rewatches": 0,
#     "avg_completion_rate_per_video": 0.0
# }
# df_final_agg = df_final_agg.fillna(numeric_fill_cols)
# df_final_agg = df_final_agg.withColumn(
#     "course_video_list",
#     F.when(F.col("course_video_list").isNull(), F.array().cast(ArrayType(StringType())))
#      .otherwise(F.col("course_video_list"))
# ).withColumn(
#     "user_video_list",
#     F.when(F.col("user_video_list").isNull(), F.array().cast(ArrayType(StringType())))
#      .otherwise(F.col("user_video_list"))
# )

# # Tính toán các chỉ số cuối cùng
# df_final_output_raw = df_final_agg.withColumn(
#     "num_common_videos",
#     F.col("num_videos_watched")
# ).withColumn(
#     "estimated_watched_in_course",
#     F.col("total_watched_duration")
# ).withColumn(
#     "completion_rate_raw", # Đặt tên thô
#     F.when(F.col("total_video_duration") > 0, F.col("total_watched_duration") / F.col("total_video_duration"))
#      .otherwise(0.0)
# ).withColumn(
#     "unique_videos_ratio_raw", # Đặt tên thô
#     F.when(F.col("num_videos_in_course") > 0, F.col("num_videos_watched") / F.col("num_videos_in_course"))
#      .otherwise(0.0)
# ).withColumn(
#     "rewatch_ratio_original",
#     F.when(F.col("num_sessions") > 0, F.col("num_rewatches") / F.col("num_sessions"))
#      .otherwise(0.0)
# ).withColumn(
#     "rewatch_sessions_ratio",
#     F.when(
#         (F.col("num_sessions") > 0) & (F.col("num_sessions") > F.col("num_videos_watched")),
#         (F.col("num_sessions") - F.col("num_videos_watched")) / F.col("num_sessions")
#     ).otherwise(0.0)
# ).withColumn(
#     "video_activity_ratio",
#     F.when(F.col("num_videos_in_course") > 0, F.col("num_sessions") / F.col("num_videos_in_course"))
#      .otherwise(0.0)
# ).withColumn(
#     "days_from_enroll_to_first_watch",
#     F.when(
#         F.col("first_local_start_time").isNotNull() & F.col("enroll_time").isNotNull(),
#         F.datediff(
#             F.col("first_local_start_time").cast("date"),
#             F.col("enroll_time").cast("date")
#         )
#     ).otherwise(None)
# )

# # --- BẮT ĐẦU TINH CHỈNH 2: CHUẨN HÓA (CAPPING) ---
# # Giới hạn các giá trị ngoại lệ (outliers) do lỗi dữ liệu
# MAX_COMPLETION = 2.0 # Giới hạn tỷ lệ hoàn thành ở 200% (cho phép xem lại 1 lần)
# MAX_RATIO = 1.0       # Giới hạn tỷ lệ video ở 100%

# df_final_output = df_final_output_raw.withColumn(
#     "completion_rate",
#     F.when(F.col("completion_rate_raw") > MAX_COMPLETION, MAX_COMPLETION)
#      .otherwise(F.col("completion_rate_raw"))
# ).withColumn(
#     "unique_videos_ratio",
#     F.when(F.col("unique_videos_ratio_raw") > MAX_RATIO, MAX_RATIO)
#      .otherwise(F.col("unique_videos_ratio_raw"))
# )
# # --- KẾT THÚC TINH CHỈNH 2 ---


# # Sắp xếp lại các cột (Giữ nguyên)
# final_columns = [
#     "user_id", "course_id", "certificate", "enroll_time",
#     "start_date", "end_date", "duration_days", "remain_day",
#     "first_local_start_time",
#     "days_from_enroll_to_first_watch",
#     "user_video_list", "avg_watched_duration", "avg_speed",
#     "num_videos_watched", "num_sessions", "total_watched_duration",
#     "course_video_list", "total_video_duration",
#     "num_common_videos", "estimated_watched_in_course", "completion_rate", # Đã chuẩn hóa
#     "unique_videos_ratio", # Đã chuẩn hóa
#     "rewatch_ratio_original",
#     "rewatch_sessions_ratio",
#     "avg_completion_rate_per_video", # Đã sửa logic
#     "video_activity_ratio"
# ]
# df_final_output = df_final_output.select(final_columns)


# # THÊM MỚI: Vòng lặp định dạng % (Thêm vào cuối)
# rate_cols = [
#     "completion_rate",
#     "unique_videos_ratio",
#     "rewatch_ratio_original",
#     "rewatch_sessions_ratio",
#     "avg_completion_rate_per_video",
#     "video_activity_ratio"
# ]

# for c in rate_cols:
#     df_final_output = df_final_output.withColumn(
#         c,
#         F.concat(F.round(F.col(c) * 100, 2).cast("string"), F.lit("%"))
#     )

# # Hiển thị kết quả (đã tinh chỉnh và chuẩn hóa)
# df_final_output.orderBy(F.col("completion_rate").desc_nulls_last()).show(20, truncate=True)

In [ ]:
# # Lọc để chỉ hiển thị những hàng có thời gian xem lớn hơn 0
# df_active_users = df_final_output.filter(F.col("total_watched_duration") > 0)

# df_active_users.show()

In [ ]:
# total = df_final_output.count()

# null_ratio = df_final_output.select([
#     (F.count(F.when(F.col(c).isNull(), 1)) / total * 100).alias(c) for c in df_final_output.columns
# ])
# null_ratio.show()

In [ ]:
# from pyspark.sql import functions as F

# # Tạo một DataFrame mới để chuẩn bị lưu CSV
# # Chuyển đổi các cột mảng thành chuỗi, phân tách bằng dấu '|'
# df_for_csv = df_final_output.withColumn(
#     "user_video_list", 
#     F.concat_ws("|", F.col("user_video_list"))
# ).withColumn(
#     "course_video_list", 
#     F.concat_ws("|", F.col("course_video_list"))
# )

# # Bây giờ tất cả các cột đều là kiểu nguyên thủy, có thể lưu ra CSV
# df_for_csv.coalesce(1).write.mode("overwrite").option("header", "true").csv("/kaggle/working/df_final_output")

# print("Đã lưu file CSV thành công!")

In [ ]:
# import shutil

# input_dir = "/kaggle/working/df_final_output"
# output_zip = "/kaggle/working/df_final_output"

# shutil.make_archive(output_zip, 'zip', input_dir)
# print("✅ Đã nén file thành df_final_output.zip")
